In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_table('./data/round1_ijcai_18_train_20180301.txt', sep=' ')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [3]:
context_data = data[['context_id', 'context_timestamp', 'context_page_id', 'predict_category_property', 'is_trade']]
context_data.head()

,context_id,context_timestamp,context_page_id,predict_category_property,is_trade
0,282924576738839389,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,0
1,4007979028023783431,1537243232,4001,5799347067982556520:9172976955054793469;790838...,0
2,4884875192608989870,1537211052,4001,5799347067982556520:5131280576272319091;725801...,0
3,840119421106178602,1537222670,4016,509660095530134768:-1;5799347067982556520:-1;7...,0
4,1736769971710354684,1537271320,4001,5799347067982556520:9172976955054793469;790838...,0


In [4]:
context_data.context_id.value_counts()

290808226164722964     3
7259819519976136023    3
393440996556281300     2
6603062751043810606    2
8100475734761363134    2
304650542434805843     2
4408407584681704919    2
3171393537650530997    2
2679502183138154630    2
1647541348423817807    2
5716302324470047044    2
4497638035869905621    2
305737608670045906     2
2849679115978568915    2
4451796609054987809    2
6005263855939584311    2
5572499327546912720    2
1969302545828967535    2
3429462095177897954    2
8957097032852481362    2
8192490307717568017    2
2909846917991242231    2
1174885052158715015    2
1081933224899281574    2
1582607348662732737    2
3405951515098261692    1
5786964585999911187    1
6634368959015546481    1
6406611827614099972    1
2225173047222438987    1
                      ..
6724145795406088160    1
6523474241755510840    1
5758712817571533573    1
3478447449724090966    1
8361681204498630276    1
2200011761089356136    1
7115954605661654791    1
5655748133890231484    1
4771149140934117127    1


In [4]:
def feature_group(data, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        data = data[[feature, 'is_trade']]
        data_feature = data[data['is_trade'] == is_trade]
        data_feature = data_feature.reset_index(drop=True)
        data_feature_grouped = data_feature.groupby([feature]).count()
        data_feature_grouped = data_feature_grouped.reset_index()
        data_feature_grouped = data_feature_grouped.sort_values(['is_trade'], ascending=False)
    return data_feature_grouped

In [5]:
def feature_log_cal(data_feature_grouped, feature, is_trade):
    if (is_trade != 1 and is_trade != 0):
        return None
    else:
        new_feature = feature + '_' + str(is_trade)
        data_feature_grouped[new_feature] = np.log(data_feature_grouped['is_trade'] / len(data_feature_grouped))
#         # 0-1标准化
#         data_feature_grouped[new_feature] =  (data_feature_grouped['is_trade'] - data_feature_grouped['is_trade'].min())/\
#                                         (data_feature_grouped['is_trade'].max() - data_feature_grouped['is_trade'].min())
    return data_feature_grouped

# 1. context_page_id

In [6]:
context_page_id_grouped_1 = feature_group(context_data, 'context_page_id', 1)
context_page_id_grouped_1 = feature_log_cal(context_page_id_grouped_1, 'context_page_id', 1)
context_page_id_grouped_1

,context_page_id,is_trade,context_page_id_1
0,4001,5246,5.569489
1,4002,924,3.832980
2,4003,586,3.377588
3,4004,460,3.135494
4,4005,335,2.818398
5,4006,260,2.564949
6,4007,228,2.433613
7,4008,187,2.235376
8,4009,142,1.960095
9,4010,122,1.808289


In [7]:
len(context_page_id_grouped_1)

20

In [8]:
context_page_id_grouped_0 = feature_group(context_data, 'context_page_id', 0)
context_page_id_grouped_0 = feature_log_cal(context_page_id_grouped_0, 'context_page_id', 0)
context_page_id_grouped_0

,context_page_id,is_trade,context_page_id_0
0,4001,262240,9.481283
1,4002,42827,7.669192
2,4003,30869,7.341775
3,4004,23628,7.074455
4,4005,18486,6.829037
5,4006,14836,6.609080
6,4007,12127,6.407457
7,4008,10209,6.235293
8,4009,8872,6.094923
9,4010,7350,5.906723


In [9]:
context_page_id_grouped_0.to_csv('./feature_data/context_data/context_page_id_grouped_0.txt', sep=' ', index=False, encoding='utf-8')
context_page_id_grouped_1.to_csv('./feature_data/context_data/context_page_id_grouped_1.txt', sep=' ', index=False, encoding='utf-8')

# 2. predict_category_property（5000多个量）

In [10]:
predict_category_property_grouped_1 = feature_group(context_data, 'predict_category_property', 1)
predict_category_property_grouped_1 = feature_log_cal(predict_category_property_grouped_1, 'predict_category_property', 1)
predict_category_property_grouped_1.head()

,predict_category_property,is_trade,predict_category_property_1
3330,7258015885215914736:-1;7822717283490579102:821...,69,-4.327104
2125,5755694407684602296:5131280576272319091;509660...,62,-4.434076
4162,7908382889764677758:-1;8896700187874717254:321...,55,-4.553877
2163,5755694407684602296:5131280576272319091;790838...,53,-4.590918
4317,8277336076276184272:-1;509660095530134768:2636...,48,-4.690009


In [11]:
predict_category_property_grouped_0 = feature_group(context_data, 'predict_category_property', 0)
predict_category_property_grouped_0 = feature_log_cal(predict_category_property_grouped_0, 'predict_category_property', 0)
predict_category_property_grouped_0.head()

,predict_category_property,is_trade,predict_category_property_0
60760,7908382889764677758:-1;8896700187874717254:321...,3065,-3.235674
63418,8277336076276184272:-1;509660095530134768:2636...,2853,-3.307350
36691,5755694407684602296:5131280576272319091;509660...,2628,-3.389498
37107,5755694407684602296:5131280576272319091;790838...,2596,-3.401749
63417,8277336076276184272:-1;509660095530134768:2636...,2351,-3.500880


In [12]:
len(predict_category_property_grouped_0) + len(predict_category_property_grouped_1)

83148

In [13]:
predict_category_property_grouped_0.to_csv('./feature_data/context_data/predict_category_property_grouped_0.txt', sep=' ', index=False, encoding='utf-8')
predict_category_property_grouped_1.to_csv('./feature_data/context_data/predict_category_property_grouped_1.txt', sep=' ', index=False, encoding='utf-8')